This code was adapted from:


*   NLP Labs


In [ ]:
pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 11.9 MB/s 


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from pandas import DataFrame
from matplotlib import pyplot
import random
import nltk
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import rand
from sklearn.metrics.pairwise import cosine_similarity
from numpy import argsort
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker
nltk.download('punkt')
nltk.download('stopwords')

drive.mount("/content/drive")

data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive
 ArtsRatings_5000_test.txt	    JewelleryReviewsLSA.csv
 ArtsRatings_5000_train.txt	    JewelleryReviewsQueryRelevantID.csv
 ArtsReviews_5000_test.txt	    metamorphosis_clean.txt
 ArtsReviews_5000_train.txt	   'My CW1.ipynb'
'F135864 - Task 1.ipynb'	   'My CW 2.ipynb'
'F135864 - Task 2.ipynb'	   'MY CW 3.ipynb'
'F135864 - Task 3.ipynb'	    Reduced_ArtsRatings_5000.txt
'F135864 - Task 4.ipynb'	    Reduced_ArtsReviews_5000.txt
'F135864 - Task 5.ipynb'	    review_polarity
 glove.6B.100d.txt		    Untitled0.ipynb
 JewelleryReviewsDeepLearning.csv


In [ ]:
# load data
file = open(data_path + "JewelleryReviewsLSA.csv",'rt')
text = file.read()
file.close()
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
# convert to lower case
tokens = [w.lower() for w in tokens]
# stemming of words
porter = PorterStemmer()
tokens = [porter.stem(word) for word in tokens]

# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]

#remove columns' titles
words = words[3:]

print(words[:100])
print(len(words))

#Correct spelling of words
spell = SpellChecker()
x = spell.unknown(words)   

for i in range (len(words)):
    # Replace each word by its correct spelling
    words[i] = spell.correction(words[i])

#check that the length of the array did not change
print(len(words))  
print(words)

['expect', 'like', 'regular', 'size', 'ring', 'thi', 'one', 'look', 'like', 'ring', 'toy', 'someth', 'funi', 'mm', 'ring', 'thi', 'ring', 'may', 'cttw', 'round', 'diamond', 'solitair', 'ring', 'white', 'gold', 'order', 'ring', 'state', 'toe', 'ring', 'descript', 'ring', 'came', 'veri', 'quickli', 'wa', 'toe', 'ring', 'nt', 'know', 'anyon', 'size', 'toe', 'know', 'anyon', 'pleas', 'direct', 'thi', 'seller', 'wed', 'band', 'toe', 'ring', 'thumb', 'ring', 'pick', 'one', 'thi', 'ring', 'beauti', 'ring', 'first', 'shipment', 'thi', 'ring', 'scratch', 'diamond', 'wa', 'miss', 'diamond', 'shoulder', 'ring', 'return', 'ring', 'replac', 'ring', 'second', 'ring', 'wa', 'even', 'wors', 'clearli', 'visibl', 'white', 'scratch', 'right', 'middl', 'black', 'diamond', 'doe', 'look', 'great', 'thi', 'time', 'diamond', 'miss', 'shoulder', 'ring', 'return', 'refund']
3963
3963
['expect', 'like', 'regular', 'size', 'ring', 'thi', 'one', 'look', 'like', 'ring', 'toy', 'someth', 'fun', 'my', 'ring', 'thi', 

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/JewelleryReviewsLSA.csv')
data_all_reviews = df["Reviews"].to_numpy()

def prepare_data(train_docs, mode, vocab):
	# encode training data set
	vectorizer = CountVectorizer(vocabulary=vocab)
	transformer = TfidfTransformer(norm='l2')
	Xtrain = transformer.fit_transform(vectorizer.fit_transform(train_docs))
	return Xtrain

# generate vocabulary
vocab = []

for ww in words:
  if ww not in vocab:
    vocab.append(ww)
#print(vocab)
# print(len(vocab))

Xtrain = prepare_data(data_all_reviews, 'tfidf', vocab)

trunc_SVD_model = TruncatedSVD(n_components=50)
approx_Xtrain = trunc_SVD_model.fit_transform(Xtrain)
print("Approximated Xtrain shape: " + str(approx_Xtrain.shape))


Approximated Xtrain shape: (200, 50)


In [ ]:

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
 
	# convert to lower case
	tokens = [word.lower() for word in tokens]

 	# stemming of words
	from nltk.stem.porter import PorterStemmer
	porter = PorterStemmer()
	tokens = [porter.stem(word) for word in tokens]

	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens


# preprocess query
def preprocess_query(review, mode, vocab):
	# clean
	tokens = clean_doc(review)
	# convert to line
	line = ' '.join(tokens)
	# encode
	vectorizer = CountVectorizer(vocabulary=vocab)
	transformer = TfidfTransformer(norm='l2')
	encoded = transformer.fit_transform(vectorizer.fit_transform([line]))
	return encoded

querys = ['The ring is a great gift. My friend loves it',
          'horrible bad quality bracelet',
					'arrived promptly and happy with the seller',
					'wear it with casual wear',
					'i expected better quality. i will return this item',
					'looks beautiful. The design is pretty. pefect and color is light',
					'This ring looks nothing like the picture. the diamonds are small and not very noticeable',
					'braclet looked just like its picture and is nice quality sterling silver.'
]
for query in querys:
  # retrieval
  encoded_query = preprocess_query(query, 'tfidf', vocab)
  # print(encoded_query.shape)

  transformed_query = trunc_SVD_model.transform(encoded_query)
  # print("Transformed query: " + str(transformed_query))
  # print("Query shape: " + str(transformed_query.shape))

  # print(type(approx_Xtrain))
  # print(approx_Xtrain)
  similarities = cosine_similarity(approx_Xtrain, transformed_query)
  # print("Similarities shape: " + str(similarities.shape))

  Top_n_reviews=10
  # indexes = np.argsort(similarities.flat)[-Top_n_reviews:]
  indexes = np.argsort(similarities.flat)[::-1]

  print('\n' + 'Query: ' + query)
  for i in range(Top_n_reviews):
    print("Top " + str(i+1) + ' result:')
    print("Review's ID: " + str(indexes[i]))
    #print(words[indexes[i]])
    # print("similarities: " + str(similarities.flat[indexes[i]]))




Query: The ring is a great gift. My friend loves it
Top 1 result:
Review's ID: 111
Top 2 result:
Review's ID: 118
Top 3 result:
Review's ID: 112
Top 4 result:
Review's ID: 66
Top 5 result:
Review's ID: 71
Top 6 result:
Review's ID: 104
Top 7 result:
Review's ID: 106
Top 8 result:
Review's ID: 117
Top 9 result:
Review's ID: 115
Top 10 result:
Review's ID: 86

Query: horrible bad quality bracelet
Top 1 result:
Review's ID: 121
Top 2 result:
Review's ID: 122
Top 3 result:
Review's ID: 124
Top 4 result:
Review's ID: 120
Top 5 result:
Review's ID: 123
Top 6 result:
Review's ID: 119
Top 7 result:
Review's ID: 194
Top 8 result:
Review's ID: 162
Top 9 result:
Review's ID: 153
Top 10 result:
Review's ID: 154

Query: arrived promptly and happy with the seller
Top 1 result:
Review's ID: 109
Top 2 result:
Review's ID: 133
Top 3 result:
Review's ID: 125
Top 4 result:
Review's ID: 153
Top 5 result:
Review's ID: 96
Top 6 result:
Review's ID: 95
Top 7 result:
Review's ID: 97
Top 8 result:
Review's ID